In [2]:
# 实现基于豆瓣top250图书评论的简单推荐系统（TF-IDF及BM25两种算法实现）

In [3]:
# 导包
import os.path as os
import csv
import jieba
from bm25_code import bm25
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
# 1、文件清洗（换行问题&空评论）
def get_format_file(oldFile, newFile):
    # 目标文件
    new_f = open(newFile, 'w')  
    with open(oldFile, 'r') as f:
        for i, line in enumerate(f.readlines()):
            # 标题行处理
            if i == 0:
                new_f.write(line + "\n")
                pre_line = ''
                continue
            # 换行问题
            if line.split("\t")[0] == pre_line.split("\t")[0]:
                new_f.write(pre_line + "\n")
                pre_line = line
            else:
                # 新书 or 评论换行
                if len(line.split("\t")) == 6:
                    # 新书
                    if pre_line.strip() != '':
                        new_f.write(pre_line + "\n")                
                    pre_line = line
                else:
                    # 评论换行
                    pre_line += line
    new_f.close()

In [5]:
# 2、文件分词
def split_file_comments(fileName):
    book_comments = {}
    with open(fileName, 'r') as f:
        # csv首行作为字典键
        lines = csv.DictReader(f, delimiter='\t')
        for line in lines:
            book_name = line.get('book', '')
            book_comment = line.get('body', '')
            
            if not book_name or not book_comment:
                continue
            
            # 分词
            comments_words = jieba.lcut(book_comment)
            
            book_comments.setdefault(book_name, []).extend(comments_words)

    
    return book_comments

In [6]:
# 3、模型训练
def comments_vectors_similarity(comments, method, stop_words):
    if method == 'bm25':
        matrix = bm25(comments, stop_words=stop_words)
    if method == 'tfidf':
        tfidf = TfidfVectorizer(stop_words=stop_words)
        matrix = tfidf.fit_transform([" ".join(comment) for comment in comments])
    
    # 计算余弦相似度
    similarity_matrix = cosine_similarity(matrix)
    
    return similarity_matrix

In [7]:
if __name__ == '__main__':
    # 1、文件清洗
    newFile = 'hw1_doubanbook_top250_comments.txt'
    oldFile = 'doubanbook_top250_comments.txt'
    if(not os.exists(newFile)):
        get_format_file(oldFile, newFile)
    
    # 2、文件分词
    book_comments = split_file_comments(newFile)
    
    # 3、模型训练
    book_name = []
    book_comment = []
    for name, comment in book_comments.items():
        book_name.append(name)
        book_comment.append(comment)
        
    stop_words = [line.strip() for line in open("stopwords.txt", "r", encoding="utf-8")]
    
    # TF-IDF算法得到的相似度矩阵
    tfidf_matrix = comments_vectors_similarity(book_comment, 'tfidf', stop_words)
    # BM25算法得到的相似度矩阵
    bm25_matrix = comments_vectors_similarity(book_comment, 'bm25', stop_words)
    
    # 4、推荐书籍
    in_book_name = '天才在左 疯子在右'
    
    book_list = list(book_comments.keys())
    in_book_idex = book_list.index(in_book_name)
    
    print(f"TF-IDF算法推荐的图书：\n")
    recommend_book_index = np.argsort(-tfidf_matrix[in_book_idex])[1:11]
    for idx in recommend_book_index:
        print(f"《{book_name[idx]}》 \t 相似度：{tfidf_matrix[in_book_idex][idx]:.4f}")
    print()
    
    print(f"BM25算法推荐的图书：\n")
    recommend_book_index = np.argsort(-bm25_matrix[in_book_idex])[1:11]
    for idx in recommend_book_index:
        print(f"《{book_name[idx]}》 \t 相似度：{bm25_matrix[in_book_idex][idx]:.4f}")
    print()

    

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\uchonor\AppData\Local\Temp\jieba.cache
Loading model cost 1.362 seconds.
Prefix dict has been built successfully.
d:\dyySoftWare\newsoftware\envs\py312\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'daren', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


TF-IDF算法推荐的图书：

《三体》 	 相似度：0.2185
《人类简史》 	 相似度：0.1930
《拆掉思维里的墙》 	 相似度：0.1889
《遇见未知的自己》 	 相似度：0.1819
《少有人走的路》 	 相似度：0.1770
《皮囊》 	 相似度：0.1720
《三体Ⅲ》 	 相似度：0.1641
《时间旅行者的妻子》 	 相似度：0.1591
《球状闪电》 	 相似度：0.1562
《穆斯林的葬礼》 	 相似度：0.1554

BM25算法推荐的图书：

《麦田里的守望者》 	 相似度：0.1044
《梦里花落知多少》 	 相似度：0.1004
《一个人的朝圣》 	 相似度：0.0953
《原来你还在这里》 	 相似度：0.0911
《何以笙箫默》 	 相似度：0.0911
《致我们终将逝去的青春》 	 相似度：0.0894
《那些回不去的年少时光》 	 相似度：0.0892
《莲花》 	 相似度：0.0861
《山楂树之恋》 	 相似度：0.0833
《平凡的世界（全三部）》 	 相似度：0.0829

